In [ ]:
#Practice problem from Analytics Vidhya. Predicting customers' purchase amounts from information about customer, products, and
# past purchases.

#Incomplete



import pandas as pd
import numpy as np
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.ensemble import VotingRegressor
from sklearn import linear_model
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import copy

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer, OneHotEncoder
from sklearn.metrics import make_scorer

import timeit
import time

#Section A: preprocessing

#lots of different parts take a long time, so this function is for keeping track of how long parts take so I can balance time
# vs. thoroughness
times = []
tstart = time.time()
def how_long(t_st):
    t1 = time.time()
    td = t1 - t_st
    times.append(td)
    print('Part {} time {}'.format(len(times), td))
    return t1

#putting data in dataframes
trainRaw = pd.read_csv('train.csv')
testRaw = pd.read_csv('test.csv')
train = trainRaw.copy()
test = testRaw.copy()

# print(train.head())
# print(test.head())

#preprocessing function
def preproc(df):
    #replace '4+' with 4 to make all Stay in City values numeric
    df['Stay_In_Current_City_Years'].replace('4+', 4, inplace=True)
    
    #convert Product Category data to strings and replace "nan" with "missing"
    missing = df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']]
    missing = missing.applymap(str)
    missing = missing.replace('nan', 'missing')
    df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']] = missing
    
    #use ordinal encoder on Gender and Age columns
    ages = ['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+']
    mapAge = list(zip(ages, range(0,7)))
    dictGend = {'col': 'Gender', 'mapping': [('M', 0), ('F', 1)]}
    oe = ce.OrdinalEncoder(cols=['Gender', 'Age'], mapping=[dictGend, {'col': 'Age', 'mapping': mapAge}])
    df = oe.fit_transform(df)
    
    #use binary encoder on categorical features that should not be ordered
    catsNotOrd = ['User_ID', 'Product_ID', 'Occupation', 'City_Category', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
    ceb = ce.BinaryEncoder(cols=catsNotOrd)
    df = ceb.fit_transform(df)
    
    #minmax scale Age
    scale = MinMaxScaler()
    df[['Age', 'Stay_In_Current_City_Years']] = scale.fit_transform(df[['Age', 'Stay_In_Current_City_Years']].values)
    
    return df

#perform preprocessing
train = preproc(train)
test = preproc(test)

#separate training data into predictive and target variables
x = train.drop('Purchase', axis=1)
y = train['Purchase']

# Trying not using some features that seem to be less important
few_feat = ['Product_ID_1', 'Product_ID_2', 'Product_ID_3', 'Product_ID_4',
       'Product_ID_5', 'Product_ID_7', 'Product_ID_8', 'Product_ID_9',
       'City_Category_2', 'Product_Category_1_1', 'Product_Category_1_2',
       'Product_Category_1_3', 'Product_Category_1_4', 'Product_Category_1_5',
       'Age', 'Stay_In_Current_City_Years']
x0 = x.copy()
test0 = test.copy()
x1 = x[few_feat]
test1 = test[few_feat]

#train-test split
xtr0, xtest0, ytr, ytest = train_test_split(x0, y, random_state=2)
xtr1, xtest1, ytr, ytest = train_test_split(x1, y, random_state=2)

#putting different versions of data into lists together
lxtr = [xtr0, xtr1]
lxtest = [xtest0, xtest1]

#timer function
endtime = how_long(tstart)


#some visualization
# f, axes = plt.subplots(3,3, figsize=(15,15))
# colCat = ['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status',
#           'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
# for a in range(0,9):
#     sns.violinplot(x=colCat[a], y='Purchase', data=trainRaw, ax=axes[int(a/3),a%3])
#
# f, ax = plt.subplots()
# sns.heatmap(trainRaw.corr())

# g = sns.PairGrid(trainRaw, y_vars="Purchase",
#                  x_vars=['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years'])
# g.map(sns.barplot)

# g = sns.PairGrid(trainRaw, y_vars="Purchase",
#                  x_vars=['Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3'])
# g.map(sns.barplot)


#Section B: building and tuning models

#record start time
tstart = time.time()

#function for calculating Root Mean Squares Logarithmic Error
def rmsle(y_targ, y_pred):
    return np.sqrt(mean_squared_error(y_targ, y_pred))

#some regressor models
rfr = RandomForestRegressor(n_estimators=20, max_depth=25, min_samples_split=0.4, max_leaf_nodes=50,
                            max_features=0.3, random_state=2, n_jobs=5, min_samples_leaf=0.15)
sgdr = linear_model.SGDRegressor(max_iter=7, random_state=2, alpha=1e-05)
gbr = GradientBoostingRegressor(learning_rate=0.2, min_samples_leaf=50, max_features='sqrt',
                                subsample=0.8, random_state=2, n_estimators=40, min_samples_split=0.6, max_depth=2)
xgbr = xgb.XGBRegressor(random_state=2, subsample=0.7, max_depth=7)

#list with models and copies to use on version of data with fewer features
models = [[rfr, sgdr, gbr, xgbr]]
models.append([copy.copy(m) for m in models[0]])

#trying to fit and score models and time how long fitting and scoring take in a systematic way

# bm = models.copy()
# bstr = [0,0,0,0]
# bste = [0,0,0,0]
# for m in range(4):
#     bm[m].fit(X_train, y_train)
#     bstr[m] = bm[m].score(X_train, y_train)
#     bste[m] = bm[m].score(X_test, y_test)
#     print(bstr[m])
#     print(bste[m])

# scores=[0,0,0,0]
# times = [0,0,0,0]
    
# def eval_score(m):
#     scores[m] = rmse_cv(models[m]).mean()
#     print(scores[m])
    
# for i in range(4):
#     times[i] = timeit.timeit('eval_score(i)', number=1, globals=globals())
#     print(times[i])

#parameters to try using Randomized Search
rfr_param_grid = {'min_samples_leaf' : np.arange(0.05, 0.36, 0.05)}
sgdr_param_grid = {'alpha': 10.0**-np.arange(1,7)}
gbr_param_grid = {'n_estimators' : range(10, 61, 10),
                  'max_depth' : range(2, 8),
                 'min_samples_split' : np.arange(0.5, 0.95, 0.05)}
xgbr_param_grid = {'max_depth' : [3, 5, 6, 7],
                  'subsample' : np.arange(0.4, 1, 0.1)}

#variables for systematically using Randomized Search on different models and data versions
#s_train = [0,0,0,0]
#s_test = [0,0,0,0]
par_grids = [rfr_param_grid, sgdr_param_grid, gbr_param_grid, xgbr_param_grid]
best_models = pd.DataFrame(index=range(4), columns=range(2))
predict_tr = best_models.copy()
predict_test = best_models.copy()

#functions for creating Randomized Search object, fitting, and using to make predictions
def makegs(model_mgs, pg_mgs):
    return RandomizedSearchCV(model_mgs, param_distributions = pg_mgs, n_iter=3, cv=3, n_jobs=5, random_state=2)

def fitgs(gsfgs, xfgs, yfgs):
    gsfgs.fit(xfgs, yfgs)
    return gsfgs

def predbm(bmpbm, xtrpbm, xtestpbm):
    ptr = bmpbm.predict(xtrpbm)
    ptest = bmpbm.predict(xtestpbm)
    return ptr, ptest

#timer function
tstart = how_long(tstart)

#i=1 is for fewer features set, not i=0
# for j in range(1,4):
#     i=0
#     gsearch = makegs(models[j], par_grids[j])
#     gsearch = fitgs(gsearch, lxtr[i], ytr)
#     tstart = how_long(tstart)
#     print(gsearch.best_params_)
#     for i in range(2):
#         best_models.iloc[j, i] = gsearch.best_estimator_

#fitting best models from Randomized Search to different versions of data
for j in range(4):
    for i in range(2):
        best_models.iloc[j, i] = models[i][j]
        best_models.iloc[j, i].fit(lxtr[i], ytr)
        tstart = how_long(tstart)

#making predictions using fitted models
for i in range(2):
    for j in range(4):
        prtr, prtest = predbm(best_models.iloc[j, i], lxtr[i], lxtest[i])
        print(prtr)
        print(prtest)
        predict_tr.iloc[j, i] = prtr
        predict_test.iloc[j, i] = prtest
        tstart = how_long(tstart)

#trying another way of doing the two previous loops
# for i in range(2):
#     for j in range(4):
#         gsearch = makegs(models[j], par_grids[j])
#         gsearch = fitgs(gsearch, lxtr[i], ytr)
#         tstart = how_long(tstart)
#         best_models.iloc[j, i] = gsearch.best_estimator_
#         print(gsearch.best_params_)
#         prtr, prtest = predbm(best_e, lxtr[i], lxtest[i])
#         print(prtr)
#         print(prtest)
#         predict_tr.iloc[j, i] = prtr
#         predict_test.iloc[j, i] = prtest
#         tstart = how_long(tstart)



#Section C: RMSE CV scores

#timer function
tstart = time.time()

#function for Cross-Validated Root Mean Squares Error
def rmse_cv(model, x_rmsecv, y_rmsecv):
    rmse = np.sqrt(-cross_val_score(model, x_rmsecv, y_rmsecv, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

#empty dataframe for RMSE scores
scores_rmsecv = pd.DataFrame(index=range(4), columns=range(2))

#RMSE scores of only the models that used the data with fewer features
print('RMSE CV scores, fewer features models only')
for j in range(4):
    scores_rmsecv[1][j] = rmse_cv(best_models[1][j], xtr1, ytr)
    print(scores_rmsecv[1][j])

#timer function
endtime = how_long(tstart)


#Section D: model scores and RMSLE scores

tstart = time.time()

#empty dataframes for scores
tr_scores_m = pd.DataFrame(index=range(4), columns=range(2))
tr_scores_rmsle = tr_scores_m.copy()
test_scores_m = tr_scores_m.copy()
test_scores_rmsle = tr_scores_m.copy()

#function for getting scores using built-in score function and RMSLE
def scores(model_sc, x_sc, y_sc, predict_sc):
    score_m = model_sc.score(x_sc, y_sc)
    score_rmsle = rmsle(y_sc, predict_sc)
    return score_m, score_rmsle

#printing results
for i in range(2):
    print('Set {}'.format(i))
    for j in range(4):
        tr_scores_m[i][j], tr_scores_rmsle[i][j] = scores(best_models[i][j], lxtr[i], ytr, predict_tr[i][j])
        test_scores_m[i][j], test_scores_rmsle[i][j] = scores(best_models[i][j], lxtest[i], ytest, predict_test[i][j])
        print('Model {}'.format(j))
        print('Training {}, {}'.format(tr_scores_m[i][j], tr_scores_rmsle[i][j]))
        print('Test {}, {}'.format(test_scores_m[i][j], test_scores_rmsle[i][j]))

endtime = how_long(tstart)


#Section E: feature importances

tstart = time.time()

#averages of feature importances
def avpr_featimp(smodels, colnames):
    df_f_imp = pd.DataFrame(index=colnames, columns=[0, 2, 3])
    feat_imp = [m.feature_importances_ for m in smodels]
    
    df_f_imp.iloc[:] = np.transpose(feat_imp)
    df_f_imp['average'] = df_f_imp.mean(axis=1)
    print(df_f_imp)
    print(df_f_imp['average'].loc[df_f_imp['average']>0.01])
    return df_f_imp

print('Feature importances')
cm0 = best_models[0].iloc[[0, 2, 3]]
f_imp0 = avpr_featimp(cm0, x0.columns)
cm1 = best_models[1].iloc[[0, 2, 3]]
f_imp1 = avpr_featimp(cm1, x1.columns)

endtime = how_long(tstart)